## Exercise 12a: Q learning with MPC-based Function Approximation for pointmass task

In this exercise, we implement Q learning for MPC-based approximation.

### Enviroment: Pointmass task
Linear dynamics defined with $A$ and $B$
Goal: Push the pointmass to the origin as shown in the following figure
<img src="imgs/pointmass.png" alt="400" width="400"/>
The baseline stage cost (reward) is \
$L(s,a) = 9x^2 + 9y^2 + \dot{x}^2 + \dot{y}^2 + F_x^2 + F_y^2$\
with the transition dynamics as 
$ s_+ = A s+ B a + bias + \mathcal{N}(0, 0.05)\\
A = \begin{bmatrix}
1 & 0 & 0.1 & 0 \\
0 & 1 & 0 & 0.1 \\
0 & 0 & 0.9 & 0 \\
0 & 0 & 0 & 0.9
\end{bmatrix},\,
B = \begin{bmatrix}
0 & 0 \\
0 & 0 \\
0.1 & 0 \\
0 & 0.1
\end{bmatrix}\,, bias = \begin{bmatrix}
0 \\
0 \\
0 \\
0
\end{bmatrix}$

### Agent: MPC-based agent
An MPC-based agent with $\theta$ learnable parameters (represented with $P$ in the exercise code)    

#### MPC formulation
- Symbolic solver definition:
    1) 'nlp_solver': gives actions ($\pi$) and value function $V$
    2) 'q_solver': gives action-value function $Q$ \
    with sensitivities $\nabla_\theta Q$, $\nabla_\theta V$ and $\nabla_\theta \pi$

- Stage cost model: Quadratic cost formulation
$L_\theta(s,a) = (s-s_{\mathrm{ref}})^T W_\theta W_\theta^T (s-s_{\mathrm{ref}}) + a^T R_\theta R_\theta^T a \\
T_\theta(s) = (s-s_{\mathrm{ref}})^T tW_\theta tW_\theta^T (s-s_{\mathrm{ref}})$ 
where $W_\theta$, $R_\theta$ and $tW_\theta$ are upper-triangular matrices.

- Transition model: state space model
    1) 'pointmass_v0_model': fixed A, B and bias
    2) 'pointmass_v1_model': learnable A, B and bias

#### MPC function approximation
Wrapper over the symbolic solver definitions.
It delivers the numerical values for $Q_\theta$, $V_\theta$ and $\pi_\theta$ with sensitivities $\nabla_\theta Q$, $\nabla_\theta V$ and $\nabla_\theta \pi$

#### Learning module 
- Q learning, the parameters $\theta$ are such that\
$\theta^\star = \arg\min_\theta \|Q^\star(s,a) - Q_\theta(s,a)\|^2$

In general, the update rule is,
$\theta \leftarrow \theta + \alpha \delta \nabla_\theta Q_\theta(s,a) \\
\delta = L(s,a) + \gamma V_\theta(s_+) - Q_\theta(s,a)$
with $\delta$ as the temporal-difference (TD) error and $\alpha$ as the learning rate.

- RL update step \
$\Delta \theta = \arg\min_{\Delta \theta} 0.5 \|\Delta \theta\|^2 + \alpha dJ^T \Delta \theta \\
\qquad \mathrm{s.t.} -0.2 \leq \Delta \theta \leq 0.2 \\ 
\theta = \theta + \Delta \theta \\
dJ = \mathbb{E}[\delta \nabla_\theta Q(s,a)]$

### Replay Buffer
Data buffer to store all the transition information $(s, a, s_+, L(s,a))$ along with some additional information (e.g. primal-dual variables, solver solutions)

### Experiment Setup

In [1]:
# Basic setup
import numpy as np
import casadi as csd

import pointmass
from mpc_formulation import MPCformulation
from mpc import MPCfunapprox
from replay_buffer import BasicBuffer, ReplayBuffer

obs_dim, action_dim = 4,2
seed = 1
env_str = "PointMass"
env_params = {
        "start_loc": [1.0, 1.0, 0.0, 0.0],
        "init_std": 0.2,
        "step_std": 0.05,
        "stochasticity": 1.0
    }
agent_params= {
        "gamma": 0.95,
        "model_str": "pointmass_v0_model",
        "opt_params": {
            "cost_defn": ["fullW", "fullR"],
            "cost_wt": [[1.0, 1.0, 1.0, 1.0], [0.1, 0.1]],
            "horizon": 10
        },
        "eps": 0.25,
        "learning_params": {
            "lr": 1e-3,
            "tr": 0.2,
            "train_params": {
                "iterations": 25,
                "batch_size": 32
            },
            "constrained_updates": True
        }
    }
n_iterations = 25
n_trains = 1
n_evals = 0
n_steps = 50
max_len_buffer = 25

# Experiment init
env = getattr(pointmass, env_str)(env_str, env_params, seed)
replay_buffer = ReplayBuffer(max_len_buffer, seed)

In [2]:
# Additional functions
def rollout_sample(env, agent, mode="train"):
    state, obs = env.reset()
    agent.reset(obs)
    rollout_return = 0
    rollout_buffer = BasicBuffer()

    for _ in range(n_steps):
        action, add_info = agent.act_forward(obs, mode=mode)
        next_state, next_obs, reward, _ = env.step(action)
        if mode == "train":
            rollout_buffer.push(
                state, obs, action, reward, next_state, next_obs, add_info
            )
        rollout_return += reward
        state = next_state.copy()
        obs = next_obs.copy()
    return rollout_return, rollout_buffer


### Q learning module

In [3]:
class Qlearning:
    def __init__(self, mpc, learning_params, seed):
        # hyperparameters
        self.mpc = mpc
        self._parse_agent_params(**learning_params)

    def train(self, replay_buffer: ReplayBuffer):
        """
        Updates Qfn parameters by using (random) data from replay_buffer

        Parameters
        ----------
        replay_buffer : ReplayBuffer object
            Class instance containing all past data

        Returns
        -------
        dict : {l_theta: parameters of Qfn,
                TD_error: observed TD_error}

        """
        td_avg = 0
        batch_size = min(self.batch_size, replay_buffer.size)
        train_it = min(self.iterations, int(3.0 * replay_buffer.size / batch_size))

        for it in range(train_it):
            (
                states,
                _,
                actions,
                rewards,
                next_states,
                _,
                infos,
            ) = replay_buffer.sample(batch_size)

            del_J = 0.0
            for j, state in enumerate(states):
                action = actions[j]
                next_state = next_states[j]
                reward = rewards[j]
                
                # Q value of the state-action pair
                q, info = ...
                # Hint: we found the Q value for state and action in exercise 11
                
                # V value of the next state
                v_next, info_next = ...

                # TD error
                td_target = ...
                
                # sensitivity of Q 
                grad_q = ...
                
                # estimate of dJ
                del_J -= td_target * grad_q.T
                td_avg += td_target
            del_J = del_J/batch_size
                
            # RL update step
            self.mpc.param_update(del_J, constrained_updates=self.constrained_updates)
        print(f"Averaged TD error: {td_avg / train_it}")

    def _parse_agent_params(self, lr, tr, train_params, constrained_updates=False):
        self.lr = lr
        self.tr = tr
        self.iterations = train_params["iterations"]
        self.batch_size = train_params["batch_size"]
        self.constrained_updates = constrained_updates


### MPC agent

In [4]:
class MPCfunapprox_ex(MPCfunapprox, MPCformulation):
    def __init__(self, agent_params, seed=1):
        # Parameters
        self._parse_agent_params(**agent_params)
        
        # Model init
        self.model = getattr(pointmass, self.model_str)()
        self.obs_dim = self.model.obs_space.shape[0]
        self.action_dim = self.model.action_space.shape[0]

        # MPC initilizaiton
        MPCformulation.__init__(self, self.model, self.opt_params, self.gamma)

        # Seeded random number generators
        self.rng1 = np.random.default_rng(seed)
        self.rng2 = np.random.default_rng(seed)
        self.rng3 = np.random.default_rng(seed)

        # Parameter values
        self.pf_val = 0.1 * self.rng1.random((self.nPf, 1))
        self.p_val = 0.1 * self.rng1.random((self.nP, 1))
        if "cost_wt" in self.opt_params:
            self.p_val[
                self.iP_cost[0] : self.iP_cost[1]
            ] = self.cost_model.cost_param_init(self.opt_params["cost_wt"])
        self.pf_val[self.iP_cost[1]:], self.p_val[self.iP_cost[1]:] = self.model.model_param_init() 
        
        # initiate the learning module
        self.learning_module = Qlearning(self, self.learning_params, seed)
        
        # RL update formulation
        self.constraint_param_opt(self.learning_module.lr, self.learning_module.tr) 

# Agent init
agent = MPCfunapprox_ex(agent_params, seed)
# Test run
_, obs = env.reset()
agent.reset(obs)
act0, info = agent.act_forward(obs)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



### Main loop

In [ ]:
# main loop
for it in range(n_iterations):
    print(f"Iteration: {it}")
    t_returns, e_returns = [],[]

    # training rollouts
    for _ in range(n_trains):
        rollout_return, rollout_buffer = rollout_sample(env, agent, mode="train")
        replay_buffer.push(rollout_buffer.buffer)
        t_returns.append(rollout_return)

    # agent training
    agent.train(replay_buffer)

    # training rollouts
    for _ in range(n_evals):
        rollout_return, rollout_buffer = rollout_sample(env, agent, mode="eval")
        e_returns.append(rollout_return)

    print(f"Training rollout return: {np.mean(t_returns)}")
    # print(f"Evaluation rollout return: {np.mean(e_returns)}")


In [ ]:
# final evaluation performance
f_returns = []
for _ in range(50):
    rollout_return, rollout_buffer = rollout_sample(env, agent, mode="final")
    f_returns.append(rollout_return)
print(f"Final rollout return: {np.mean(f_returns)}")

### Environment 
Pointmass with penalty
The baseline stage cost (reward) is \
$L(s,a) = 9x^2 + 9y^2 + \dot{x}^2 + \dot{y}^2 + F_x^2 + F_y^2 - 100(x)_-$\
where function $(x)_-$ gives $1$ for $x<0$ and $0$ otherwise 
with the transition dynamics as 
$ s_+ = A s+ B a \\
A = \begin{bmatrix}
1 & 0 & 0.1 & 0 \\
0 & 1 & 0 & 0.1 \\
0 & 0 & 0.9 & 0 \\
0 & 0 & 0 & 0.9
\end{bmatrix},\,
B = \begin{bmatrix}
0 & 0 \\
0 & 0 \\
0.1 & 0 \\
0 & 0.1
\end{bmatrix}$

In [10]:
obs_dim, action_dim = 4,2
seed = 1
env_str = "PointMass_v2"
env_params = {
        "start_loc": [1.0, 1.0, 0.0, 0.0],
        "init_std": 0.2,
        "step_std": 0.05,
        "stochasticity": 1.0
    }
agent_params= {
        "gamma": 0.95,
        "model_str": "pointmass_v1_model",
        "opt_params": {
            "cost_defn": ["fullW", "fullR"],
            "cost_wt": [[3.0, 3.0, 1.0, 1.0], [0.1, 0.1]],
            "horizon": 10
        },
        "eps": 0.25,
        "learning_params": {
            "lr": 1e-3,
            "tr": 0.2,
            "train_params": {
                "iterations": 25,
                "batch_size": 32
            },
            "constrained_updates": True
        }
    }
n_iterations = 25
n_trains = 1
n_evals = 0
n_steps = 50
max_len_buffer = 25

# Experiment init
env = getattr(pointmass, env_str)(env_str, env_params, seed)
replay_buffer = ReplayBuffer(max_len_buffer, seed)

# Agent init
agent = MPCfunapprox_ex(agent_params, seed)
# Test run
_, obs = env.reset()
agent.reset(obs)
act0, info = agent.act_forward(obs)

In [6]:
# intial evaluation performance
f_returns = []
for _ in range(50):
    rollout_return, rollout_buffer = rollout_sample(env, agent, mode="final")
    f_returns.append(rollout_return)
print(f"Final rollout return: {np.mean(f_returns)}")

Final rollout return: 329.120165521956


In [ ]:
# main loop
for it in range(n_iterations):
    print(f"Iteration: {it}")
    t_returns, e_returns = [],[]

    # training rollouts
    for _ in range(n_trains):
        rollout_return, rollout_buffer = rollout_sample(env, agent, mode="train")
        replay_buffer.push(rollout_buffer.buffer)
        t_returns.append(rollout_return)

    # agent training
    agent.train(replay_buffer)

    # training rollouts
    for _ in range(n_evals):
        rollout_return, rollout_buffer = rollout_sample(env, agent, mode="eval")
        e_returns.append(rollout_return)

    print(f"Training rollout return: {np.mean(t_returns)}")
    # print(f"Evaluation rollout return: {np.mean(e_returns)}")


Iteration: 0
Averaged TD error: 18.59746249626181
W:
[[ 8.97566696e+00 -5.25952718e-03 -4.74652202e-03 -3.64586666e-03]
 [-5.25952718e-03  9.00668607e+00 -5.52288134e-03 -4.53666698e-03]
 [-4.74652202e-03 -5.52288134e-03  1.01686225e+00  7.16747164e-03]
 [-3.64586666e-03 -4.53666698e-03  7.16747164e-03  1.01298510e+00]]
R:
[[0.01030852 0.00012809]
 [0.00012809 0.01033718]]
Training rollout return: 151.49555474218224
Iteration: 1
Averaged TD error: 18.245576359469958
W:
[[ 9.35350435  0.1463097  -0.01611557 -0.01734096]
 [ 0.1463097   9.23330091 -0.01336855 -0.01525203]
 [-0.01611557 -0.01336855  1.02705189  0.0132616 ]
 [-0.01734096 -0.01525203  0.0132616   1.03082454]]
R:
[[0.01100069 0.00043643]
 [0.00043643 0.01107205]]
Training rollout return: 228.93660352271147
Iteration: 2
Averaged TD error: 72.40766495518866
W:
[[ 9.35167784  0.10830941 -0.02250943 -0.0155444 ]
 [ 0.10830941  9.14453354 -0.01554479 -0.01451006]
 [-0.02250943 -0.01554479  1.04651537  0.01536353]
 [-0.0155444  -0.

In [8]:
# final evaluation performance
f_returns = []
for _ in range(50):
    rollout_return, rollout_buffer = rollout_sample(env, agent, mode="final")
    f_returns.append(rollout_return)
print(f"Final rollout return: {np.mean(f_returns)}")

Final rollout return: 245.94948161465084


In [9]:
agent.p_val.T

array([[ 4.27581731e+00,  2.32415188e-02,  8.49401950e-03,
        -1.90558372e-01,  3.75468524e+00, -8.78927010e-02,
         8.43673290e-02,  1.54605889e+00, -7.48767630e-02,
         1.15271979e+00,  4.37588170e-01, -7.08080747e-02,
         3.77777403e-01,  4.23950721e-02,  1.91320412e-01,
        -1.53600000e-01,  7.35917206e-02,  1.01341193e+01,
        -3.36687437e-01, -1.17109603e+00, -2.48377838e-01,
        -2.99408819e-01,  1.04844181e+01, -5.48184767e-01,
        -5.59863194e-01,  8.17428294e-01,  5.48630058e-01,
         9.21534419e+00,  2.19652348e-01, -4.31050648e-01,
         2.02197103e+00,  4.15806706e-02,  9.45116262e+00,
         3.70918986e-01, -1.61129534e-01,  1.76168264e+00,
        -5.43348995e-02, -5.90276543e-01,  2.12054831e-01,
        -1.06661684e-01,  1.73232946e+00, -1.12572179e+00,
         1.20848297e+00, -6.52769466e-01, -2.94267713e-01]])